In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from multiprocessing import Pool
import FinanceDataReader as fdr
import requests, lxml
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from user_agent import generate_user_agent, generate_navigator
import time, datetime

In [8]:
def getInvesting(ticker):
    target_address = 'https://www.google.com/search?q='+ticker+'+investing'
    html = requests.get(target_address)
    soup = BeautifulSoup(html.text, 'html.parser')
    posts = soup.select('.kCrYT a ')
    #for chooes first herf 
    href = str(posts[0].attrs['href'])
    #then href is 'https:/www.investing/equities/stock&~~~~~~ 
    #so 1.find  start-index : index('equities') , end-index : index('&')
    # and slicing href[start : end]
    start = href.find('equities')
    end = href.find('&')
    # print(href[start:end])
    return "https://www.investing.com/" + href[start:end] + '-news'

In [9]:
getInvesting('AAPL')

'https://www.investing.com/equities/apple-computer-inc-news'

In [4]:
def get_title(corp):
    comments = []
    dates = []

    page = 0
    cnt = 0
    WAIT = 10
    
    # webdriver options
    options = webdriver.ChromeOptions()
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    
    user_agent = generate_user_agent(os=('mac', 'linux'), device_type='desktop')
    options.add_argument(str(user_agent))        
    
    driver = webdriver.Chrome(executable_path='./chromedriver', options=options)
    
    # open browser
    driver.get(getInvesting(corp))
    driver.implicitly_wait(WAIT)
    
    # seledium으로 크롤링 -> 서버 관릭자가 싫어함
    # articles = driver.find_elements_by_class_name('comment_content__AvzPV')
    
    # bs4로 크롤링
    soup = BeautifulSoup(driver.page_source, 'lxml')
    
    # comment
    for line in soup.select('div div div.comment_content__AvzPV'):
        sentence = line.text
        comments.append(sentence)
    
    # date
    for line in soup.select('div.comment_chat-header__36SjM span span'):
        sentence = line.text
        dates.append(sentence)
    
    # 2page부터 짝수 page crawling
    while True:
        page += 2
        
        if page % 50 == 0:
            time.sleep(40)
        
        url = getInvesting(corp)+ '/' + str(page)            

        options = webdriver.ChromeOptions()

        # user_agent = generate_user_agent(os=('mac', 'linux'), device_type='desktop')
        user_agent = generate_user_agent(device_type='desktop')
        options.add_argument(str(user_agent))

        driver = webdriver.Chrome(executable_path='./chromedriver', options=options)

        driver.get(url)
        driver.implicitly_wait(WAIT)
        time.sleep(3)

        # articles = driver.find_elements_by_class_name('comment_content__AvzPV')
        soup = BeautifulSoup(driver.page_source, 'lxml')        
        link = soup.find_all('link', rel='canonical')
        
        # end 조건
        if link[0]['href'] == getInvesting(corp):
            print('Count comment page: ', cnt)            
            break


       # comment
        for line in soup.select('div div div.comment_content__AvzPV'):
            sentence = line.text
            comments.append(sentence)

        # date
        for line in soup.select('div.comment_chat-header__36SjM span span'):
            sentence = line.text
            dates.append(sentence)

        cnt += 1
        
    driver.close()

    return comments, dates

In [5]:
def save_csv(corporate):
    comments, dates = get_title(corporate)

    df = pd.DataFrame(
        {'DATE' : dates,
        'COMMENT' : comments})

    df.to_csv(f'{corporate}.csv', index=False, encoding='UTF-8')

In [4]:
getInvesting('AOS')

'https://www.investing.com/equities/a.o-smith-corp-commentary'

In [100]:
save_csv('AAPL')

/var/folders/xw/rbcyfp692g3c5w_0qcls7ncm0000gn/T/ipykernel_83867/3556084359.py:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='./chromedriver', options=options)
/var/folders/xw/rbcyfp692g3c5w_0qcls7ncm0000gn/T/ipykernel_83867/3556084359.py:53: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='./chromedriver', options=options)


IndexError: list index out of range